# 题目

> 给定员工的 schedule 列表，表示每个员工的工作时间。  
每个员工都有一个非重叠的时间段  Intervals 列表，这些时间段已经排好序。  
返回表示所有员工的共同、正数长度的空闲时间的有限时间段的列表，同样需要排好序。

# 方法一：优先队列

> 所有员工的每项工作构成了代办工作的总集合。  
对于每个员工来说，其代办工作都是按时间顺序排列的，因此使用一个小顶堆 pq 来维护每个员工的当前代办工作。这个小顶堆的长度等于员工的个数，小顶堆中的元素是每个员工的当前代办工作，小顶堆的队头是所有员工当前代办工作中起始时间最早的那项工作。  
使用 anchor 记录当前时间， anchor 初始化为最早代办工作的起始时间。  
按起始时间升序遍历全部代办工作：  
1. 若当前工作的起始时间 t 在当前时间 anchor 之后，由于当前工作的起始时间是剩余代办工作中最早的，因此 anchor 到 t 之间的时间是所有员工的剩余时间；  
2. 然后，若当前工作的结束时间在 anchor 之后，将时间向后推移， anchor 更新为这个工作的结束时间；  
3. 最后，当前工作办完后，在小顶堆 pq 的元素中删除当前工作，并将该员工的下一项代办工作（若有）按顺序插入小顶堆 pq 。

## 复杂度

- 时间复杂度: $O(nlogm)$ ，其中 $n$ 是字符串 s 的长度， $m$ 是不同字符的个数。

> 要遍历 n 个字符，每一次都要进行一个插入堆的操作，复杂度为 $O(nlogm)$ （因为堆的长度最多是 m ）。

- 空间复杂度: $O(m)$ ，其中 $m$ 是不同字符的个数。

> 主要是堆、双端队列的开销。

## 代码

In [1]:
class Interval:
    def __init__(self, start = None, end = None):
        self.start = start
        self.end = end

In [2]:
import heapq

In [3]:
class Solution:
    def employeeFreeTime(self, schedule):
        
        ans = []
        
        # ei表示第几名员工，emp为员工的工作时间段列表
        # emp[0].start表示员工第一项代办工作的起始时间
        pq = [(emp[0].start, ei, 0) for ei, emp in enumerate(schedule)]
        
        # 将工作时间列表重构为小顶堆
        # 其中的元素为：(最新待办工作的起始时间,员工编号,工作项数)
        heapq.heapify(pq)
        
        # 用anchor记录当前时间
        # 初始化：遍历所有员工的所有工作的起始时间，找出其中的最小值，记为anchor的初始值
        anchor = min(iv.start for emp in schedule for iv in emp)
        
        # 按起始时间由早到晚的顺序遍历每个员工的每项工作
        while pq:
            # 弹出所有代办工作中起始时间最早的一项
            t, e_id, e_jx = heapq.heappop(pq)  # t：员工最新代办工作的起始时间；e_id：第几名员工；e_jx：该员工的第几项工作
            
            # 由于anchor只在某项工作结束时更新，因此其可以代表某工作的结束时间
            # 如果最新代办工作的起始时间t在anchor之后，说明在anchor到t这个时间段所有员工都是空闲的
            if anchor < t:
                ans.append(Interval(anchor, t))
            
            # 若当前员工的当前工作结束时间大于anchor，更新anchor的值
            anchor = max(anchor, schedule[e_id][e_jx].end)
            
            # 如果当前员工还有工作没干完
            if e_jx + 1 < len(schedule[e_id]):
                # 将当前工作删除（已干完）
                # pq中的元素更新为(下一项待办工作的起始时间,员工编号,工作项数)
                heapq.heappush(pq, (schedule[e_id][e_jx+1].start, e_id, e_jx+1))

        return ans

#### 测试一

In [4]:
work1_1 = Interval(1, 2)
work1_2 = Interval(5, 6)
work2_1 = Interval(1, 3)
work3_1 = Interval(4, 10)
schedule = [[work1_1, work1_2], [work2_1], [work3_1]]

test = Solution()
ans = test.employeeFreeTime(schedule)
while ans:
    freetime = ans.pop(0)
    print([freetime.start, freetime.end])

[3, 4]


#### 测试二

In [5]:
work1_1 = Interval(1, 3)
work1_2 = Interval(6, 7)
work2_1 = Interval(2, 4)
work3_1 = Interval(2, 5)
work3_2 = Interval(9, 12)
schedule = [[work1_1, work1_2], [work2_1], [work3_1, work3_2]]

test = Solution()
ans = test.employeeFreeTime(schedule)
while ans:
    freetime = ans.pop(0)
    print([freetime.start, freetime.end])

[5, 6]
[7, 9]
